In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", temperature=0.1, 
    streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
    )

poet_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a poet who writes about programming language. Main words in poet come from programming language. You write the poet in 5 to 10 lines and it has humor "),
    ("human", "I want to make {programming} language poet")
])

comment_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are the best poetry commentator and use easy words when explaining"),
    ("human", "{poet}")
])

poet_chain = poet_prompt | chat 
comment_chain = comment_prompt | chat 

final_chain = {"poet" : poet_chain} | comment_chain
final_chain.invoke({
    "programming" : "Python"
}) 

In the land of Python, where whitespace reigns,
Indentation is key, no curly braces or chains.
With lists and tuples, dictionaries and sets,
We code with ease, no need to fret.

But beware the dreaded syntax error's sting,
For even the mightiest coder must debug and bring
Their code back to life, with a simple fix,
In the world of Python, where bugs play tricks.This poem talks about coding in Python,
Where spaces and tabs are what you rely on.
No curly braces or chains to be seen,
Just lists, dictionaries, and sets in between.

But watch out for syntax errors, they can be mean,
Even the best coders have to intervene.
Fixing bugs is part of the game,
In the Python world, where errors are tamed.

AIMessageChunk(content='This poem talks about coding in Python,\nWhere spaces and tabs are what you rely on.\nNo curly braces or chains to be seen,\nJust lists, dictionaries, and sets in between.\n\nBut watch out for syntax errors, they can be mean,\nEven the best coders have to intervene.\nFixing bugs is part of the game,\nIn the Python world, where errors are tamed.')